# AllTissueJunctionAnalysis as a Notebook 

rMATS 3.2.5 was run on controlled access RNASeq files retrieved experiments stored in the Sequence Read Archive with controlled access managed by dbGaP.   The data were generated under the Gene Tissue Expression.

## rMATS RNASeq-MATS.py produces 10 different output types which get assembled into as type junction ID by sample ID matrices

### Alternative Splice Site Types are: (se, a3ss, a5ss, mxe, ri)

 This is input as ARGV1 into variable 'astype'

  * Skipped Exon events (se),
  * Alternative 3' splice site (a3ss),
  * Alternative 5' splice site (a5ss),
  * Mutually exclusive exon (mxe),
  * and retention intron (ri)

### There are two different kinds of junction counts

  * jc = junction counts - reads that cross the junction
  * jcec = junction counts plus reads on the target (such as included exon

### And the count type -- there are 5 types

  * inclusion levels (percent spliced in)
  * included junction counts (ijc)
  * skipped junction counts (sjc)
  * inclusion length (inclen)
  * skipped length (skiplen)

### function: fit_iso_tissue 

fit_iso_tissue expects the following input:

  * the tissue of interest (SMSTD) 
  * an ordered_merged_rmats -- which will be ordered to fit the count matrix
  * count matrix (inc or ijc & sjc merged)
  * splice type (a3ss, a5ss, mxe, ri or se)
  * junction_count type (jc or jcec)
  * count type (inc or the merged ijc,sjc)
  
### reordering to match annotations between count matrix and annotation matrix

Common problem is to match specifically the rows of an annotation matrix with the columns of a count matrix
`match` is the function that gives the re-ordering index required to accomplish this


## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## rMATS-final-merged
the rmats-nf NextFlow was executed and the results released here:

## Loading dependencies

In [1]:
# temporary hack remove me when the dependencies are fixed
#
#install.packages("BiocManager")
#Sys.setenv(TAR = "/bin/tar")
#BiocManager::install(c('limma','edgeR', 'statmod'))
#install.packages(c('doParallel', 'doRNG', 'foreach', 'stringi', 'pheatmap'), repo = 'https://cran.r-project.org')
#devtools::install_github("ropensci/piggyback@87f71e8", upgrade="never")
#install.packages("runjags", repos = "https://cran.r-project.org")
#devtools::install_github("easystats/report")


In [2]:
library(limma)
library(piggyback)
library(multtest)
library(Biobase)
library(edgeR)
library(tibble)
#install.packages('R.utils')
library(R.utils)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bio

## Modeling

This analysis uses edgeR.  From the documentation, it is important to note that normalization takes the form of correction factors that enter into the statistical model. Such correction factors are usually computed internally by edgeR functions, but it is also possible for a user to supply them. The correction factors may take the form of scaling factors for the library sizes, such as computed by calcNormFactors, which are then used to compute the effective library sizes. 

Alternatively, gene-specific correction factors can be entered into the glm functions of edgeR as offsets. In the latter case, the offset matrix will be assumed to account for all normalization issues, including sequencing depth and RNA composition.

Note that normalization in edgeR is model-based, and the original read counts are not themselves transformed. This means that users should not transform the read counts in any way before inputing them to edgeR. For example, users should not enter RPKM or FPKM val- ues to edgeR in place of read counts. Such quantities will prevent edgeR from correctly estimating the mean-variance relationship in the data, which is a crucial to the statistical strategies underlying edgeR. Similarly, users should not add artificial values to the counts before inputing them to edgeR.

edgeR is not designed to work with estimated expression levels, for example as might be output by Cufflinks. 
edgeR can work with expected counts as output by RSEM, but raw counts are still preferred. 

As instructed by the software, we are using the raw counts as provided by rMATS.  The raw counts we are using in the model are `ijc` and `sjc`, the sample specific raw read counts as they align to the junctions of the `included exon (ijc)` and the junctions of the `excluded or skipped exon (sjc)` respectively.


Be sure to set your GITHUB_TOKEN, prior to downloading files

One suggestion is change it to your token and then run it then immediately change it back to this:

Sys.setenv(GITHUB_TOKEN = "your-very-own-github-token")

In [3]:
# devtools::install_github("ropensci/piggyback@87f71e8", upgrade="never")


### Did you remember?
Did you remember to delete your private github token?  Now is a good time to do so, before you save your work and checkit in inadvertantly....

In [4]:
if (!("SraRunTable.txt.gz" %in% list.files("../data/"))) {
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "TheJacksonLaboratory/sbas", 
        file = "SraRunTable.txt.gz",
        tag  = "GTExV8.v1.0", 
        dest = "../data/")
    
    message("Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..\n")   
    metadata <- data.table::fread("../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz")
    message("done!")
} else {
    message("Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..\n")   
    metadata <- data.table::fread("../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz")
    message("done!\n")
}

if (!("rmats_final.se.jc.ijc.txt.gz" %in% list.files("../data/"))) {    
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.ijc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../data/")
    message("Loading ijc counts from rmats_final.se.jc.ijc.txt.gz ../data/gtex.rds ..\n")   
    ijc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.ijc.txt.gz")
    message("done!\n")
} else {
    message("Loading ijc counts from rmats_final.se.jc.ijc.txt.gz ../data/gtex.rds ..\n")   
    ijc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.ijc.txt.gz")
    message("done!\n")    
}

if (!("rmats_final.se.jc.sjc.txt.gz" %in% list.files("../data/"))) {
    message("Downloading rmats_final.se.jc.sjc.txt.gz")
    piggyback::pb_download(
        show_progress = TRUE,
        repo = "adeslatt/sbas_test", 
        file = "rmats_final.se.jc.sjc.txt.gz",
        tag  = "rMATS.3.2.5.GTEx.V8.final_matrices", 
        dest = "../data/")
    message("Loading sjc counts from rmats_final.se.jc.sjc.txt.gz ../data/gtex.rds ..\n")   
    sjc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.sjc.txt.gz")
    message("done!\n")    

} else {
    message("Loading sjc counts from rmats_final.se.jc.sjc.txt.gz ../data/gtex.rds ..\n")   
    sjc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.sjc.txt.gz")
    message("done!\n")        
}

if (!("gtex.rds" %in% list.files("../data/"))) {
    message("Downloading and loading obj with GTEx v8 with 'yarn::downloadGTExV8()'\n")
    obj <- yarn::downloadGTExV8(type='genes',file='../data/gtex.rds')
    message("Done!\n")

} else {
# Load with readRDS() if gtex.rds available in data/
    message("Loading obj GTEx v8 rds object with readRDS from ../data/gtex.rds ..\n")   
    obj <- readRDS(file = "../data/gtex.rds")
    message("Done!\n")
    message("Generating sha256sum for gtex.rds ..\n")    
    message(system("sha256sum ../data/gtex.rds", intern = TRUE))
    message("Done!\n")
} 
if (! (file.exists("../data/fromGTF.tar.gz"))) {}
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching fromGTF.tar.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "fromGTF.tar.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_gtf",
                           tag  = "rMATS.3.2.5.gencode.v30",
                           show_progress = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.tar.gz into ../data")
    system("mkdir -p ../data && tar xvfz ../data/fromGTF.tar.gz -C ../data", intern = TRUE)
    message("Done!\n")
    message("Decompressing fromGTF.*.txt.gz into ../data")
    system("gunzip  ../data/fromGTF*.txt.gz ", intern = TRUE)
    message("Done!\n")
    message("Reading fromGTF.SE.txt into fromGTF.SE")    
    fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)}
    message("Done!\n")
} else {
        message("Reading fromGTF.SE.txt into fromGTF.SE")    
        fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)}
        message("Done!\n")
    
}

Loading metadata from SraRunTable.txt.gz ../data/gtex.rds ..

done!

Loading ijc counts from rmats_final.se.jc.ijc.txt.gz ../data/gtex.rds ..

done!

Loading sjc counts from rmats_final.se.jc.sjc.txt.gz ../data/gtex.rds ..

done!

Loading obj GTEx v8 rds object with readRDS from ../data/gtex.rds ..

Done!

Generating sha256sum for gtex.rds ..

18e2c7a83c98dcf59ddab53e1281923979d49da6ea3acb68114c5a44057c57bc  ../data/gtex.rds
Done!



### keep only those values for which we have phenotype data 
Samples were resequenced, which is shown since we have the previous step saw no reduction in information - but we are using the yarn function to correct for errors in the GTEx data set.

To deal with this, we make the metadata unique per sample, rather than unique per run -- there are a little over 100 runs that are more than one sequencing run for a sample.

There are in fact 69 samples which have more than 1 run, 67 samples that have 3 sequencing runs and 2 samples that have 2 sequencing runs.

### Now we need to reduce the metadata to the unique samples
We will use the first occurance of the sample, got the idea from here https://stackoverflow.com/questions/19944334/extract-rows-for-the-first-occurrence-of-a-variable-in-a-data-frame

In [66]:
metadata$SAMPID   <- gsub('-','\\.',metadata$'Sample Name')
pData(obj)$SAMPID <- gsub('-','\\.',pData(obj)$SAMPID)

length(unique(pData(obj)$SAMPID))
length(unique(metadata$SAMPID))
length(metadata$SAMPID)

# the runs are unique in the metadata file
length(unique(metadata$Run))
length(metadata$Run)

# 69 samples which have more than 1 run, 
# 2 samples that have 2 sequencing runs and 
# 67 samples that have 3 sequencing runs.
# as the following three commands illustrate
# we use t (which is the number of runs per sample) as the index
# to reduce the table
unique_index <- unique(metadata$SAMPID)
t <- table(metadata$SAMPID)
table((t[t>1]))
names_gt_1 <- names(table(t[t>1]))
length(unique_index)

metadata_first <- metadata[match(unique(metadata$SAMPID), metadata$SAMPID),]
dim(metadata_first)

# now that we have this reduced we can merge these together
metadata_pData <- dplyr::left_join(metadata_first, pData(obj), by = "SAMPID")
dim(metadata_pData)

[1] 17383

[1] 9648

[1] 9784

[1] 9784

[1] 9784


 2  3 
 2 67 

[1] 9648

[1] 9648   80

[1] 9648  146

### Name the rows by the SE Junction ID

Name the rows with the junction id column and then make the matrix just about the counts.
Adjust the matrix, preparing them to be pure count matrices

In [67]:
# preserve junction id as rowname
rownames(ijc.iso.counts.mem) <- ijc.iso.counts.mem$ID
rownames(sjc.iso.counts.mem) <- sjc.iso.counts.mem$ID

# and remove the id to have a data matrix
ijc.iso.counts.mem2  <- ijc.iso.counts.mem[,-1]
sjc.iso.counts.mem2  <- sjc.iso.counts.mem[,-1]

dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)

[1] 42611  8673

[1] 42611  8673

### Line up the run names in the metadata with the count matrices
We need to do two things, one refer to only those runs that we have appropriate metadata for and two make sure the order of both are in sync.  First, we make sure that both the metadata runs and the ijc, sjc counts are the same

In [68]:
ijc_run_names <- as.vector(as.character(colnames(ijc.iso.counts.mem2)))
run_names     <- as.vector(as.character(metadata_pData$Run))

non_overlaps <- setdiff( ijc_run_names, run_names)

message("The non-overlaping IDs between pheno and count data are:\n\n", 
        paste(length(non_overlaps), collapse = "\n") )

ijc_in_metadata =run_names %in% ijc_run_names

length(ijc_in_metadata)
table(ijc_in_metadata)

reduced_metadata_pData <- metadata_pData[ijc_in_metadata==TRUE,]

ijc_run_names      <- as.vector(as.character(colnames(ijc.iso.counts.mem2)))
reduced_run_names  <- as.vector(as.character(reduced_metadata_pData$Run))

non_overlaps <- setdiff( ijc_run_names, reduced_run_names)

message("The non-overlapping IDs between ijc, sjc count matrix and the reduced_metadata are:\n\n", 
        paste(length(non_overlaps), collapse = "\n") )

runs_in_ijc  <- ijc_run_names %in% reduced_run_names 
table(runs_in_ijc)
length(runs_in_ijc)

ijc.iso.counts.mem3 <- data.matrix(ijc.iso.counts.mem2)
sjc.iso.counts.mem3 <- data.matrix(sjc.iso.counts.mem2)
dim(ijc.iso.counts.mem3)
dim(sjc.iso.counts.mem3)

ijc.iso.counts.mem3 <- ijc.iso.counts.mem3[,runs_in_ijc==TRUE]
sjc.iso.counts.mem3 <- sjc.iso.counts.mem3[,runs_in_ijc==TRUE]

dim(ijc.iso.counts.mem3)
dim(sjc.iso.counts.mem3)
dim(reduced_metadata_pData)

The non-overlaping IDs between pheno and count data are:

119


[1] 9648

ijc_in_metadata
FALSE  TRUE 
 1094  8554 

The non-overlapping IDs between ijc, sjc count matrix and the reduced_metadata are:

119


runs_in_ijc
FALSE  TRUE 
  119  8554 

...

[1] 42611  8673

[1] 42611  8673

[1] 42611  8554

[1] 42611  8554

[1] 8554  146

###  Order ijc and sjc columns in the same order as the metadata Run order

Using tibble library, we can rearrange the columns as the column name.  

In [69]:
metadata_runnames    <- as.character(reduced_metadata_pData$Run)
metadata_SAMPID      <- as.character(reduced_metadata_pData$SAMPID)

ijc.iso.counts.mem4  <- as_tibble(ijc.iso.counts.mem3)
sjc.iso.counts.mem4  <- as_tibble(sjc.iso.counts.mem3)

ijc.iso.counts.mem4  <- ijc.iso.counts.mem4[,c(metadata_runnames)]
sjc.iso.counts.mem4  <- sjc.iso.counts.mem4[,c(metadata_runnames)]

dim(ijc.iso.counts.mem4)
dim(sjc.iso.counts.mem4)
dim(reduced_metadata_pData)
write.csv(reduced_metadata_pData,"../data/reduced_metadata_pData.csv")

[1] 42611  8554

[1] 42611  8554

[1] 8554  146

### Removing 11IL0...

Remove samples that match '11IL0' from the ijc, sjc and metadata files using the logical grep, grepl

In [70]:
keep_metadata <- (!grepl('11ILO',reduced_metadata_pData$SAMPID))
table(keep_metadata)
ijc.iso.counts.mem4 <-ijc.iso.counts.mem4      [                    ,keep_metadata==TRUE]
sjc.iso.counts.mem4 <-sjc.iso.counts.mem4      [                    ,keep_metadata==TRUE]
reduced_metadata_pData <-reduced_metadata_pData[keep_metadata==TRUE,                    ]

dim(reduced_metadata_pData)
dim(ijc.iso.counts.mem4)
dim(sjc.iso.counts.mem4)

keep_metadata
FALSE  TRUE 
   12  8542 

[1] 8542  146

[1] 42611  8542

[1] 42611  8542

### and now for all tissues


### exploration of the details

For each sample, we have ijc and sjc count data and demo
For exon skipping events (SE), we have 42,611 non-zero junction IDs the (first dimension of the ijc and sjc cout table) for the skipped exon event for breast-Mammary Tissue, 191 individuals.  These are healthy individuals, and we are studying the impact of sex on the occurrence or non-occurance of specific alternative splicing events.   We explore the information we ahve about these junctions and create a construct, as_event, which accounts for the junction under exploration.

#### Exploring the ijc and sjc Count data 

We have two counts that are in many ways two sides of the same coin.  Both our the observational output and we wish to see how robust each are in their ability to separate out the samples to provide for us differentially expressed isoform events as measured by their counts.   Each junction is in a manner a specific marker to specific isoform events that may or may not be shared between the genders.   If there is significant results, then this is indicative of the separation achieved by isoform specific differentiation.   In our model we will use these in combination, it is important to see if they will yield the results we are looking for.

### What else could we add to our linear model
There are a couple of other items we can add to the mix, to make the data a bit more interesting
We will use the yarn SEX phenotype 1==male, 2==female, rather than the SRA sex phenotype.

In [71]:
table(reduced_metadata_pData$AGE)
table(reduced_metadata_pData$DTHHRDY)
table(reduced_metadata_pData$SMCENTER)
table(reduced_metadata_pData$SEX)
table(reduced_metadata_pData$sex)


20-29 30-39 40-49 50-59 60-69 70-79 
  624   558  1362  2625  2467   123 


   0    1    2    3    4 
4169  397 1942  407  762 


        B1     B1, A1         C1     C1, A1 C1, B1, A1         D1     D1, A1 
      4669        369       2113        491          0         60         57 


   1    2 
4938 2821 


female   male 
  3218   5324 

## Preparing the data further

### Keeping only tissues shared male female

We need to remove the tissues that are not shared by males and females, we do this by finding the intersection of the tissue lists.

In [72]:
# SEX is coded 1 == Male
#              2 == Female
sex = factor(reduced_metadata_pData$SEX)
sex2 <- ifelse(sex == 1,'male','female')
table(sex)
table(sex2)
tissue_reduction <- read.table(file="../assets/tissues.tsv", header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
colnames(tissue_reduction)  <- c("SMTSD","female","male","include","display_name")

# only include those tissues we wish to continue with
table(tissue_reduction$include)
tissue_reduction <- tissue_reduction[tissue_reduction$include==1,]


# reduced the expressionSet object to only those tissues we wish to keep
# create a matching tissue name to go with the expressionSet phenotype object
reduced_metadata_pData$tissue <- factor(snakecase::to_snake_case(as.character(reduced_metadata_pData$SMTSD)))
tissue_reduction$SMTSD        <- factor(snakecase::to_snake_case(as.character(tissue_reduction$SMTSD)))
                                                          
tissue_names_keep <- reduced_metadata_pData$tissue %in% tissue_reduction$SMTSD
table(tissue_names_keep)
length(tissue_names_keep)
dim(reduced_metadata_pData)

reduced_metadata_pData<-reduced_metadata_pData[tissue_names_keep==TRUE,                       ]
ijc.iso.counts.mem5   <-ijc.iso.counts.mem4   [                       ,tissue_names_keep==TRUE]
sjc.iso.counts.mem5   <-sjc.iso.counts.mem4   [                       ,tissue_names_keep==TRUE]
dim(reduced_metadata_pData)
dim(ijc.iso.counts.mem5)
dim(sjc.iso.counts.mem5)

#table(reduced_metadata_pData$tissue)

sex
   1    2 
4938 2821 

sex2
female   male 
  2821   4938 


 0  1 
11 39 

tissue_names_keep
FALSE  TRUE 
 1609  6933 

[1] 8542

[1] 8542  147

[1] 6933  147

[1] 42611  6933

[1] 42611  6933

### Create sex specific tissues


In [74]:

male_tissues_true   <- sex2 == "male"
female_tissues_true <- sex2 == "female"

tissue_list         <- reduced_metadata_pData$tissue
male_tissue_list    <- factor(reduced_metadata_pData$tissue[male_tissues_true   == TRUE])
female_tissue_list  <- factor(reduced_metadata_pData$tissue[female_tissues_true == TRUE])

male_female_tissue_list <- intersect(levels(male_tissue_list),levels(female_tissue_list))

keep = tissue_list %in% male_female_tissue_list

table(keep)

ijc_m_f         = ijc.iso.counts.mem5    [          ,keep==TRUE]
sjc_m_f         = sjc.iso.counts.mem5    [          ,keep==TRUE]
metadata_m_f    = reduced_metadata_pData2[keep==TRUE,          ]
tissue_list_m_f = tissue_list            [keep==TRUE]

dim(ijc_m_f)
dim(sjc_m_f)
dim(metadata_m_f)
length(tissue_list_m_f)
tissue_list_m_f <- factor(tissue_list_m_f)
levels(tissue_list_m_f)
length(levels(tissue_list_m_f))

keep
TRUE 
6933 

[1] 42611  6933

[1] 42611  6933

[1] 6933  147

[1] 6933

[1] "adipose_subcutaneous"                 
 [2] "adipose_visceral_omentum"             
 [3] "adrenal_gland"                        
 [4] "artery_aorta"                         
 [5] "artery_coronary"                      
 [6] "artery_tibial"                        
 [7] "brain_caudate_basal_ganglia"          
 [8] "brain_cerebellar_hemisphere"          
 [9] "brain_cerebellum"                     
[10] "brain_cortex"                         
[11] "brain_frontal_cortex_ba_9"            
[12] "brain_hippocampus"                    
[13] "brain_hypothalamus"                   
[14] "brain_nucleus_accumbens_basal_ganglia"
[15] "brain_putamen_basal_ganglia"          
[16] "brain_spinal_cord_cervical_c_1"       
[17] "breast_mammary_tissue"                
[18] "cells_cultured_fibroblasts"           
[19] "cells_ebv_transformed_lymphocytes"    
[20] "colon_sigmoid"                        
[21] "colon_transverse"                     
[22] "esophagus_gastroesophageal_junction"  
[23] "esophagus_mucosa"                     
[24] "esophagus_muscularis"                 
[25] "heart_atrial_appendage"               
[26] "heart_left_ventricle"                 
[27] "liver"                                
[28] "lung"                                 
[29] "muscle_skeletal"                      
[30] "nerve_tibial"                         
[31] "pancreas"                             
[32] "pituitary"                            
[33] "skin_not_sun_exposed_suprapubic"      
[34] "skin_sun_exposed_lower_leg"           
[35] "small_intestine_terminal_ileum"       
[36] "spleen"                               
[37] "stomach"                              
[38] "thyroid"                              
[39] "whole_blood"

[1] 39

### Keeping only chromosomes shared male female

The Y chromosome spans more than 59 million base pairs of DNA and represents almost 2 percent of the total DNA in cells. Each person normally has one pair of sex chromosomes in each cell. The Y chromosome is present in males, who have one X and one Y chromosome, while females have two X chromosomes. Since our analysis is on the comparative differences, we must eliminate chrY from our analyses.

To do so, we grab the annotation from the GTF file and remove those junctions that correspond to the genes on this chromosome

In [ ]:
head(fromGTF.SE)
genes <- factor(fromGTF.SE$geneSymbol)
length(levels(genes))    

table(fromGTF.SE$chr)

keepAllJunctionsButChrY <- (fromGTF.SE$chr != "chrY")

table(keepAllJunctionsButChrY)
sum(table(keepAllJunctionsButChrY))

fromGTF_no_chrY <- fromGTF.SE[keepAllJunctionsButChrY,]
ijc_m_f_no_chrY <- ijc_m_f   [keepAllJunctionsButChrY,]
sjc_m_f_no_chrY <- sjc_m_f   [keepAllJunctionsButChrY,]

dim(ijc_m_f_no_chrY)
dim(sjc_m_f_no_chrY)
dim(fromGTF_no_chrY)


ijc_m_f_no_chrY <- data.matrix(ijc_m_f_no_chrY)
sjc_m_f_no_chrY <- data.matrix(sjc_m_f_no_chrY)

rownames(ijc_m_f_no_chrY) <- rownames(fromGTF_no_chrY)
rownames(sjc_m_f_no_chrY) <- rownames(fromGTF_no_chrY)

head(ijc_m_f_no_chrY)
head(fromGTF_no_chrY)
head(sjc_m_f_no_chrY)

## Exploratory and Differential analysis as_event:ijc, sjc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the included exon junction count expression; sex denotes the reported sex of the subject

## Differential analysis as_event (combined ijc and sjc)

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + B2 as_event + B3 sex*as_event + epsilon (error)
           

where y is the alternative splicing event expression; sex denotes the reported sex of the subject, as_event represents the specific alternative splicing event - either included exon junction counts or skipped exon junction counts and their interaction terms.   Donor is added to our model as a blocking variable used in both the calculation of duplicate correlation as well as in the linear fit.

### Voom, limma's lmFit and eBayes

Using sample as a blocking variable, we are able to model the effects of the donor on the results, which improves the power.  This topic is discussed in biostars https://www.biostars.org/p/54565/.  And Gordon Smyth answers the question here https://mailman.stat.ethz.ch/pipermail/bioconductor/2014-February/057887.html.  The method of modeling is a random effects approach in which the intra-donor correlation is incorporated into the covariance matrix instead of the linear predictor.   And though as Gordon Smyth states both are good method and the twoway anova approach makes fewer assumptions, the random effects approach is statistically more powerful.  

We have a balanced design in which all donors receive all stimuli (which is really in healthy human donors, life and all of its factors!) Our measurement has so many points -- we are measuring in the skipped exon approach, 42,611 junctions!   It is not possible to encorporate those measurements into the linear predictor.  A two-way ANOVA approach is virtually as powerful as the random effects approach 
and hence is preferable as it makes fewer assumptions.

For an unbalanced design in which each donor receives only a subset of the stimula, the random effects approach is more powerful.

Random effects approach is equivalent to The first method is twoway anova, a generalization of a paired analysis.


In [ ]:

cat(levels(tissue_list_m_f),sep="\n")

actual_tissue_list_m_f = levels(tissue_list_m_f)
tissue_of_interest = actual_tissue_list_m_f[21]
tissue_of_interest
length(actual_tissue_list_m_f)
length(tissue_list_m_f)

dim(ijc_m_f_no_chrY)
dim(sjc_m_f_no_chrY)
dim(metadata_m_f)
dim(fromGTF_no_chrY)

tissue_of_interest = tissue_of_interest
fromGTF = fromGTF_no_chrY
tissue_list = tissue_list_m_f
ijc = ijc_m_f_no_chrY
sjc = sjc_m_f_no_chrY
metadata =  metadata_m_f


In [ ]:

cat(levels(tissue_list_m_f),sep="\n")

actual_tissue_list_m_f = levels(tissue_list_m_f)
tissue_of_interest = actual_tissue_list_m_f[21]
tissue_of_interest
length(actual_tissue_list_m_f)
length(tissue_list_m_f)

dim(ijc_m_f_no_chrY)
dim(sjc_m_f_no_chrY)
dim(metadata_m_f)
dim(fromGTF_no_chrY)

print_exploratory_plots (tissue_of_interest, 
                         fromGTF_no_chrY, 
                         tissue_list_m_f, 
                         ijc_m_f_no_chrY, 
                         sjc_m_f_no_chrY, 
                         metadata_m_f )


## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
rm (notebookid)
notebookid   = "AllTissueJunctionAnalysis"
notebookid

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

paste0("../metadata/", notebookid, "_sha256sums.txt")

data.table::fread(paste0("../metadata/", notebookid, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]